Программа обучает многослойный перцептрон (MLP) — простую нейронную сеть — решать XOR-проблему, то есть различать точки двух классов, которые нелинейно разделимы (их нельзя разделить одной прямой).

### В коде создаются четыре группы точек на плоскости:

<img src="output/source_data.png" width="600">

Далее выборка перемешивается и делится на обучающую (75%) и тестовую (25%):

In [ ]:
split_point = int(num_samples * 0.75)
train_data, train_labels = data[train_idx], labels[train_idx]
test_data,  test_labels  = data[test_idx],  labels[test_idx]

### Архитектура сети

Создаём модель:

In [ ]:
model = MLP(
    layer_sizes=[2, 10, 10, 1],
    activation_name=RELU,
    learning_rate=0.1
)

| Слой        | Кол-во нейронов | Назначение                      |
| ----------- | --------------- | ------------------------------- |
| Входной     | 2               | координаты (x₁, x₂)             |
| 1-й скрытый | 10              | ищет внутренние закономерности  |
| 2-й скрытый | 10              | уточняет и комбинирует признаки |
| Выходной    | 1               | выдаёт вероятность класса (0–1) |

activation_name=RELU — используем ReLU в скрытых слоях.

learning_rate=0.1 — шаг обучения (скорость изменения весов).


### Обучение 

Эпоха — один полный проход по всем обучающим данным.

Здесь их 10 000, что означает 10 000 итераций обновления весов сети.


1. Сеть делает прямой проход (forward) — вычисляет предсказания.

2. Считается ошибка (MSE = среднеквадратичное отклонение).

3. Делается обратное распространение (backward) — вычисляется градиент ошибки по весам.

4. Весы и смещения обновляются на маленький шаг (learning_rate).

Выводится в консоли

Epoch 1000/10000 | Loss: 0.032145 | Accuracy: 96.75%

- Loss — значение функции потерь (насколько сеть ошибается). Чем меньше — тем лучше.

- Accuracy — точность классификации (% правильных ответов).

- Вывод идёт каждые print_interval эпох (т.е. каждые 1000 эпох).

#### forward

In [ ]:
Z = W @ A + b
A = activation(Z)

Z — взвешенная сумма входов

A — выход после активации (ReLU или Sigmoid)

Последний слой — линейный, но затем результат пропускается через сигмоиду для получения вероятности [0, 1].

#### backward

Здесь считается градиент ошибки и обновляются веса

In [ ]:
dZ = dA * derivative(activation(Z))
weights[l] -= learning_rate * dW
biases[l] -= learning_rate * dB

| Функция     | Формула                          | Диапазон | Назначение                                                 |
| ---| --- | --- | --- |
| **Sigmoid** | $$ \sigma(x) = 1 / (1 + e^{-x}) $$ | (0, 1)   | Превращает любое число в "вероятность"                     |
| **ReLU**    | $$ f(x) = \max(0, x) $$          | [0, ∞)   | Пропускает только положительные сигналы, ускоряет обучение |
| **Tanh**    | $$ \tanh(x) $$                     | (-1, 1)  | Часто для скрытых слоёв                                    |


### Предсказания и графики после обучения

In [ ]:
predictions = model.predict(X_test).reshape(-1)
pred_colors = ['blue' if p > 0.5 else 'red' for p in predictions]
plt.scatter(test_data[:,0], test_data[:,1], c=pred_colors)

Каждая точка теста окрашена:

- Синяя, если сеть считает класс 1

- Красная, если сеть считает класс 0

Если обучение успешно, цвета на этом графике должны соответствовать исходным:

левый нижний и правый верхний — красные

левый верхний и правый нижний — синие.

<img src="output/predicted_data.png" width="600">

### Ошибки классификации

<img src="output/errors.png" width="600">

почти вся плоскость правильно окрашена, а чёрные точки лежат вдоль границы между красными и синими областями, именно по краю классов модель сомневается

#  Немного (очень много) теории 

**Перцептро́н** — это математическая или компьютерная модель, которая имитирует то, как мозг обрабатывает информацию.  
Каждый нейрон принимает входные данные, взвешивает их по важности и решает, «включиться» или «молчать».

---

##  Структура сети

Перцептроны соединяются в **слои**.  
Каждый слой — это группа нейронов, работающих параллельно.

### 🔹 Входной слой (Input Layer)
Получает исходные данные — например, координаты X и Y.

### 🔹 Скрытые слои (Hidden Layers)
Обрабатывают информацию и находят закономерности.  
Именно здесь происходит «мышление» нейросети.

### 🔹 Выходной слой (Output Layer)
Выдаёт результат — например, вероятность того, что точка принадлежит классу 1.

Каждый слой **преобразует входы** и передаёт результаты дальше.  
Скрытые слои создают новые **признаки (feature maps)**, которые делают задачу “более линейной” для следующего уровня.

---

##  Пример рассуждений сети

- Первый слой может “заметить”, что `x` и `y` по-разному большие или маленькие.  
- Второй слой комбинирует эти признаки и выделяет “XOR-области”.  
- Выходной слой выдаёт итоговую вероятность — например, 0.93 (то есть 93% уверенности, что это класс 1).

---

##  Краткий словарь понятий

| Понятие | Что это |
|:--|:--|
| **Перцептрон** | Математическая модель нейрона |
| **Вес (weight)** | Показывает, насколько важен вход |
| **Bias (смещение)** | Сдвигает функцию, помогает подстроить решение |
| **Функция активации** | Делает поведение нейрона нелинейным |
| **Слой** | Группа нейронов, выполняющих преобразование |
| **MLP (многослойный перцептрон)** | Сеть из нескольких слоёв, способная учить сложные зависимости |
| **Обучение** | Подгонка весов по ошибке с помощью обратного распространения |

---

##  Что такое функция активации

**Функция активации** — это «решатель» нейрона.  
Она определяет:

> «Нужно ли нейрону активироваться (включиться) и насколько сильно».

Каждый нейрон получает сигнал на входе — сумму взвешенных входов:

$$
z = w_1 x_1 + w_2 x_2 + b
$$

Это просто число — например, $z = 3.7$ или $z = -1.2$.  
Но если передавать его дальше **без изменений**, сеть останется **линейной**,  
а линейная модель не умеет решать сложные задачи вроде XOR.

Чтобы добавить **нелинейность**, применяют функцию активации:

$$
a = f(z)
$$

где:  
- $z$ — это вход нейрона (сумма сигналов),  
- $f$ — функция активации,  
- $a$ — выход нейрона (то, что он передаёт дальше).

---

##  Зачем она нужна

Без функции активации сеть работала бы как обычная линейная формула:  

$$
y = W_2 (W_1 x) = (W_2 W_1) x
$$

Сколько слоёв ни добавь — результат всё равно линейный.  
Такая сеть **никогда не решит XOR**, потому что XOR — это **нелинейная зависимость**.

---

##  Что делает активация

Функция активации **искажает** (или **сжимает**) входные данные,  
добавляя в поведение нейрона нелинейность.  
Благодаря этому сеть может:

- “гнуться” и подстраиваться под сложные закономерности,  
- разделять запутанные области на плоскости,  
- делать решения плавными, а не резкими.

---

##  Как ведёт себя нейрон

-  При **малом входе** нейрон почти «молчит» (выдаёт ≈ 0)  
-  При **большом входе** — «включается полностью» (≈ 1)  
-  В промежутке — **частично активируется**

Так сеть учится распознавать плавные переходы и не реагировать на шум.

---

 Итого:  
Функция активации — это именно тот элемент, который делает нейросеть «умной».  
Без неё перцептрон был бы просто калькулятором.

## 📈 Примеры функций активации

Теперь давай посмотрим, какие бывают и зачем каждая нужна.

---
<img src="output/output.png" width="800">

###  **1. Сигмоида (sigmoid)**  

**Формула:**
$$
f(x) = \frac{1}{1 + e^{-x}}
$$

 Преобразует любое число в диапазон от **0 до 1**.

| x  | f(x)  |
|----|--------|
| -5 | ≈ 0.00 |
|  0 | 0.50 |
|  5 | ≈ 1.00 |

 **Интерпретация:**
- маленькие значения (отрицательные) → нейрон «спит» (почти 0)
- большие значения (положительные) → нейрон «включён» (почти 1)
- около нуля → «частично активен»

 **Поэтому:** сигмоида отлично подходит для **выходного слоя**, где нужно получить вероятность (например, принадлежность к классу 1).

 **Минусы:**
- “залипает” на крайних значениях (градиенты почти 0),
- обучение становится медленным (градиент исчезает).

---

###  **2. ReLU (Rectified Linear Unit)**  

**Формула:**
$$
f(x) = \max(0, x)
$$

 То есть:
- если `x > 0`, возвращаем `x`,
- если `x <= 0`, возвращаем `0`.

| x  | f(x) |
|----|------|
| -2 | 0 |
|  0 | 0 |
|  2 | 2 |

 **Интерпретация:**
- Нейрон **включается** только если сигнал положительный,  
- И **передаёт его “как есть”**.

 Это как выключатель:  
если вход меньше 0 — “молчи”,  
если больше — “говори”.

 **Плюсы:**
- Очень быстро считается,  
- Не даёт градиентам исчезать (в отличие от сигмоиды),  
- Хорошо работает в глубоких сетях.

 **Минус:**
- Если нейрон “уйдёт” в отрицательную область, он **навсегда выключится** (градиент = 0) — это называют “проблемой мёртвых нейронов”.

---

###  **3. tanh (гиперболический тангенс)**  

**Формула:**
$$
f(x) = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$

 Диапазон от **–1 до 1**, более “сбалансированная” версия сигмоиды.

| x  | f(x) |
|----|------|
| -3 | -1 |
|  0 | 0 |
|  3 | 1 |

 Часто используют в скрытых слоях, потому что значения около 0 помогают нейронам обучаться стабильнее.

---

##  Почему именно такие

Каждая функция выбрана не просто так:

| Функция | Где используется | Почему |
|----------|------------------|--------|
| **Sigmoid** | Выходной слой (для вероятности) | Даёт числа от 0 до 1 |
| **ReLU** | Скрытые слои | Быстрая, не даёт исчезать градиенту |
| **tanh** | Иногда в скрытых | Центрирована вокруг 0, стабильнее сигмоиды |

---

##  Пример с аналогией

Нейрон — это **лампочка с диммером** (регулятором яркости):

| Входной ток (z) | Функция активации | Что делает |
|------------------|--------------------|-------------|
| z = -2 | Sigmoid → 0.12 | Лампочка почти не светит |
| z = 0  | Sigmoid → 0.5  | Светит наполовину |
| z = 2  | Sigmoid → 0.88 | Почти полная яркость |
| z = 2  | ReLU → 2       | Светит в 2 раза ярче |
| z = -2 | ReLU → 0       | Полностью выключена |

То есть активация **управляет “светимостью” нейрона** — насколько сильно он передаёт сигнал дальше.

---

##  Итого: коротко и просто

| Название | Диапазон | Что делает | Где лучше |
|-----------|-----------|-------------|------------|
| **Sigmoid** | 0 … 1 | “Сжимает” выход в вероятность | Выходной слой |
| **ReLU** | 0 … ∞ | “Включает” только положительные значения | Скрытые слои |
| **tanh** | –1 … 1 | “Балансирует” сигналы | Иногда скрытые |
